# Ensemble with Meta Model = LSTM, CNN, ATT, TF
Create: 06/12/2022
Description: Ensemble - Combination of Data and Staking Model by LSTM, CNN, ATT, TF. Data from 1997-2015. 44 validation
Modification: 31/01/2023

In [1]:
!nvidia-smi

Thu Feb 16 00:00:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:21:00.0 Off |                  Off |
| 34%   54C    P2   112W / 260W |   3611MiB / 24576MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install packages
# !pip install -U scikit-learn
# !pip install ftfy
# !pip install xlsxwriter 
# !pip install xlwt

# Imports

In [3]:
# Imports
import tensorflow as tf 
# from google.colab import drive
import os

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense

import traceback
import sys
import copy
import numpy as np
from itertools import combinations
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

# lib for LSTM of Meta model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
import seaborn as sns


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

from ftfy import fix_text
import warnings
warnings.simplefilter("ignore", UserWarning)

plt.rcParams['figure.figsize'] = [7, 3]
import math

# Global param and project path

In [4]:
# Attach Google Drive for reading and saving files
# drive.mount('/content/drive')
prj_path = '/home/mlworker/Quang/HealthCare/Source_14012023_v4/'

path_all_mix_data =             prj_path+ 'results/results_14022023/all_mix_data/'
path_all_mix_data_cnn =         prj_path+ 'results/results_14022023/all_mix_data/cnn/'
path_all_mix_data_lstm =        prj_path+ 'results/results_14022023/all_mix_data/lstm/'
path_all_mix_data_lstm_att =    prj_path+ 'results/results_14022023/all_mix_data/lstm_att/'
path_all_mix_data_tf =          prj_path+ 'results/results_14022023/all_mix_data/tf/'

prj_path_opt= prj_path+ 'optimize_hyperparam/opt_results/opt_results_1997_2015_mae/'

prj_path_result_ensemble = prj_path+ 'results/ensemble_15022023/'
com_path =      prj_path+ 'results/ensemble_15022023/result/simple_combination/all_combinations_15.xlsx'
com_path_2 =    prj_path+ 'results/ensemble_15022023/result/simple_combination/all_combinations_25.xlsx'
com_path_3 =    prj_path+ 'results/ensemble_15022023/result/simple_combination/all_combinations_35.xlsx'
com_path_4 =    prj_path+ 'results/ensemble_15022023/result/simple_combination/all_combinations_45.xlsx'

com_path_metrics =      prj_path+ 'results/ensemble_15022023/result/simple_combination/all_combinations_metrics_15.xlsx'
com_path_metrics_2 =    prj_path+ 'results/ensemble_15022023/result/simple_combination/all_combinations_metrics_25.xlsx'
com_path_metrics_3 =    prj_path+ 'results/ensemble_15022023/result/simple_combination/all_combinations_metrics_35.xlsx'
com_path_metrics_4 =    prj_path+ 'results/ensemble_15022023/result/simple_combination/all_combinations_metrics_45.xlsx'



# os.chdir(prj_path)

# loockback_window for meta model. default = 3 months for alls
meta_lookback_window = 3
len_validation = 24 # validation or test depends on usecase

#Note: mỗi bộ all_mix_data sẽ có lenght = 251 (1997 -01/01/2018  và vì 1 dòng đầu bị bỏ đi do quá trình statitionary và remove NAN lúc train)
lenght_whole_data_city = 215

# Xây dựng Meta Data Set

In [5]:
##########################
# Xây Dựng Meta Data Set
meta_test_set_city = {}
meta_train_set_city = {}
meta_data_set_city = {}
cities = [
#         'An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
#         'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
          
#         'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
#         'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
          
#         'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
#         'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
          
#         'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
#         'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
          
        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
        'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên', 
        'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp'
         ]

# cities = ['Đà Nẵng']

for city in cities:  
  meta_test_set = pd.DataFrame()
  meta_train_set = pd.DataFrame() 
  
  
  ######################################
  # Xử lý data của CNN ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_cnn+'mix_train_test_data_'+city+'_cnn.xlsx')
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index() 
  # Tạo meta testset
  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-len_validation:-len_validation] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
  meta_test_cnn = result_test.reset_index(drop=True)  
  # Tạo meta trainset
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-len_validation]
  meta_train_cnn = result_train.reset_index(drop=True)
  
  ######################################
  # Xử lý data của LSTM  ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_lstm+'mix_train_test_data_'+city+'_lstm.xlsx')  
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-len_validation:-len_validation] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
  meta_test_lstm = result_test.reset_index(drop=True)
  # Tạo meta trainset
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-len_validation]
  meta_train_lstm = result_train.reset_index(drop=True)

  ###########################################
  # Xử lý data của LSTM ATT ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_lstm_att+'mix_train_test_data_'+city+'_lstm_att.xlsx') 
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  # Meta testset sẽ có lenght = 36 + lookback_window   
  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-len_validation:-len_validation] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
  meta_test_lstm_att = result_test.reset_index(drop=True)  
  
  # Tạo meta trainset  
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-len_validation]
  meta_train_lstm_att = result_train.reset_index(drop=True)

  # lấy observation
  meta_test_observed = selected_result['Observed'].iloc[-meta_lookback_window-len_validation:]
  meta_test_observed = meta_test_observed.reset_index(drop=True) 
  meta_train_observed = selected_result['Observed'].iloc[meta_lookback_window:-len_validation]
  meta_train_observed = meta_train_observed.reset_index(drop=True)

  #############################################
  # # Xử lý data của Transformer ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_tf+'mix_train_test_data_'+city+'_tf.xlsx') 
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  # Meta testset sẽ có lenght = 36 + lookback_window 
  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-len_validation:-len_validation] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation)) # Lấy 36 tháng cuối
  meta_test_tf = result_test.reset_index(drop=True)  
  
  # Tạo meta trainset
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-len_validation]
  meta_train_tf = result_train.reset_index(drop=True)

  
  meta_test_set['cnn_pred'] = meta_test_cnn
  meta_test_set['lstm_pred'] = meta_test_lstm
  meta_test_set['lstm_att_pred'] = meta_test_lstm_att
  meta_test_set['tf_pred'] = meta_test_tf
  meta_test_set['Observed'] = meta_test_observed
   
  meta_train_set['cnn_pred'] = meta_train_cnn
  meta_train_set['lstm_pred'] = meta_train_lstm
  meta_train_set['lstm_att_pred'] = meta_train_lstm_att 
  meta_train_set['tf_pred'] = meta_train_tf  
  meta_train_set['Observed'] = meta_train_observed

  meta_test_set_city[city] = meta_test_set # test set có chiều dài 39
  meta_train_set_city[city] = meta_train_set # train set có chiều dài 212 = 252 -36 (train) - 1 (station) - 3 (meta_lookback_window)

  # Đoạn này làm thêm. Nối dài meta_test_set_city và meta_train_set_city
  df = meta_train_set.append(meta_test_set[meta_lookback_window:])
  df.to_excel(prj_path_result_ensemble+'/meta_data/meta_data_'+city+'.xlsx')  

<ipython-input-5-988b42e80534>:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
<ipython-input-5-988b42e80534>:54: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
<ipython-input-5-988b42e80534>:70: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
<ipython-input-5-988b42e80534>:93: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_resu

<ipython-input-5-988b42e80534>:93: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation)) # Lấy 36 tháng cuối
<ipython-input-5-988b42e80534>:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = meta_train_set.append(meta_test_set[meta_lookback_window:])
<ipython-input-5-988b42e80534>:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
<ipython-input-5-988b42e80534>:54: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['T

<ipython-input-5-988b42e80534>:39: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
<ipython-input-5-988b42e80534>:54: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
<ipython-input-5-988b42e80534>:70: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(len_validation))
<ipython-input-5-988b42e80534>:93: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_resu

In [6]:
# meta_train_set_city['An Giang']

In [7]:
# meta_test_set_city['An Giang']

# Danh sách các based Model và các Combination có thể của nó

In [8]:
# Danh sách các based Model và các Combination có thể của nó
list_based_model = ['cnn_pred','lstm_att_pred','lstm_pred','tf_pred']

combs_features = list(combinations(list_based_model[0:], 2)) + list(combinations(list_based_model[0:], 3)) + list(combinations(list_based_model[0:], 4))


# list_based_model = ['cnn_pred','lstm_att_pred','lstm_pred','tf_pred']

# combs_features = list(combinations(list_based_model[0:], 4)) 
combs_features

[('cnn_pred', 'lstm_att_pred'),
 ('cnn_pred', 'lstm_pred'),
 ('cnn_pred', 'tf_pred'),
 ('lstm_att_pred', 'lstm_pred'),
 ('lstm_att_pred', 'tf_pred'),
 ('lstm_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_att_pred', 'lstm_pred'),
 ('cnn_pred', 'lstm_att_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_pred', 'tf_pred'),
 ('lstm_att_pred', 'lstm_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_att_pred', 'lstm_pred', 'tf_pred')]

In [9]:
# combs_features = [('cnn_pred', 'lstm_att_pred')]

# Support Functions and Train class 

In [10]:
def seed_everything(seed: int):
    import random, os    
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)



# Set hyperparameters as args using the Configuration class
class Configuration():
    def __init__(self):
      # lấy bộ test dài 36 tháng = 3 năm
        self.test_size = len_validation
        # là nhìn vào dữ liệu trước 3 tháng và dự phóng        
        self.look_back = 3
        # dự phóng 3 tháng
        self.n_predicted_month = 3
        self.n_features = 3
        self.seed = 42
        # mỗi phần tử x trong tập suppervise có độ lớn là 16 = 16 tháng
        self.batch_size = 16
        # self.device = torch.device("cuda")
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.epochs = 300
     

# Biến args lưu giá trị Global.
args = Configuration()


# Lưu ý sau khi stationnary thì dòng đầu tiên trừ cho dòng trước nữa là ko có cho nên về cơ bản sẽ thành NaN
# Cần remove dòng đầu tiên ra khỏi data để train và test. Cho nên cơ bản là data sau khi in ra sẽ thiếu 1 đơn vị đầu tiên
def convert_to_stationary(city_data):
    """Subtracts previous value for all cols except disease rates"""
    for col_name in city_data.columns:
        if col_name != 'Diarrhoea_rates':
            try:
                city_data[col_name] = city_data[col_name] - city_data[col_name].shift()
            except:
                print(col_name)
    return city_data

def impute_missing_value(city_data):
    """
    Cơ bản dữ liệu bị thiếu sót rất nhiều: Như Điện Biên 1997 -2003 là thiếu dữ liệu về bệnh
    Hàm này sẽ tự sinh ra dữ liệu bị thiếu. Nếu tháng nào không có số liệu thì tính như sau:
    12 tháng đầu không có số liệu thì gán = 0
    tháng 13-24 không có số liệu, sẽ lấy giá trị của tháng cùng kỳ năm trước
    tháng từ 24 trở đi sẽ lấy giá trị nhỏ nhất của 2 tháng cùng kỳ trong 2 năm gần nhất.
    Do Điện Biên bằng 0 nên sau khi xử lý từ 1997 -2003 là đều = 0.  
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(city_data[col].iloc[index - 12], city_data[col].iloc[index - 24])
    return city_data

def split_data(data, look_back ):
    """Splits data into train and test sets based on args (Configuration class)"""
    train = data[: -args.test_size]    
    test = data[-args.test_size - look_back: ]
    return train, test

def to_supervised(data, d_in=args.look_back, d_out=args.n_predicted_month, features_list=[]):
    """
    Frames time-series as supervised learning dataset.
    
    Args:
      d_in: lookback window
      d_out: number of predicted months
      features_list: list of all features **where last col is the disease incidence**

    Returns:
      Numpy arrays of disease incidence (y) and other predictors (X)
    """
    X, y = list(), list()
    for index, _ in enumerate(data):
        in_end = index + d_in
        out_end = in_end + d_out
        if out_end <= len(data):
            if len(features_list) == 0 :
                X.append(data[index: in_end, :])
            else:
                X.append(data[index: in_end, features_list])
            y.append(data[in_end: out_end, -1])
    return np.array(X), np.array(y)


def get_data(train_np, test_np, batch_size, list_selected_features, origin_city_meta_data):
    """
    Returns important feature list and data formatted for input into Pytorch 
    models
    """ 
    # origin_city_meta_data chứa data mẫu để lấy index 
    # important_features = arrary index nên ta phải truyền indexex của columns
    important_features = [origin_city_meta_data.columns.get_loc(c) for c in list_selected_features if c in origin_city_meta_data]  

    train_X, train_y = to_supervised(train_np, features_list=important_features)
    test_X, test_y = to_supervised(test_np, features_list=important_features)
    train_tensor = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
    test_tensor = (torch.from_numpy(test_X), torch.from_numpy(test_y))

    train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

    return important_features, train_loader, test_tensor

#Define Pytorch LSTM model
class MultiVariateLSTM(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = output[:, -1, :]
        y_predicted = self.linear(last_hidden_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            #return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

#Define Pytorch LSTM-ATT model
class MultiVariateLSTM_Attention(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM_Attention, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.attention_linear = nn.Linear(hidden_size, hidden_size)
        # self.linear = nn.Linear(hidden_size*2, args.n_predicted_month)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = last_hidden[-1]
        remain_hidden_vector = output
        e_t = remain_hidden_vector.bmm(self.attention_linear(last_hidden_vector).unsqueeze(2)).squeeze(-1)
        alpha_t = F.softmax(e_t, dim=1)
        attenion_vector = remain_hidden_vector.transpose(2, 1).bmm(alpha_t.unsqueeze(2)).squeeze(-1)
        # combine_vector = torch.cat((last_hidden_vector, attenion_vector), dim=1)
        # combine_vector = last_hidden_vector + attenion_vector
        y_predicted = self.linear(attenion_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Define Pytorch CNN model
class MultivariateCNN(nn.Module):
    def __init__(self, num_filters=[100, 100, 100], dropout=0.01):
        super(MultivariateCNN, self).__init__()
        self.loss_fn = loss = nn.MSELoss()
        self.filter_sizes = [1, 2, 3]
        self.conv1d_list = nn.ModuleList([nn.Conv1d(args.n_features, num_filters[i], self.filter_sizes[i]) for i in range(len(self.filter_sizes))])
        self.linear = nn.Linear(np.sum(num_filters), args.n_predicted_month)
        self.dropout = nn.Dropout(dropout)
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))

    def forward(self, X_batch, y_batch=None):
        X_batch = X_batch.permute(0, 2, 1)  #(batch_size, n_features, n_look_back)
        X_conv_list = [F.relu(conv1d(X_batch)) for conv1d in self.conv1d_list]
        X_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in X_conv_list]
        X_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in X_pool_list], dim=1)
        y_predicted = self.linear(self.dropout(X_fc))
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted

    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Transformer Model
# Transformer Model
class PositionalEncoder(nn.Module):
    def __init__(self, n_feature=3, dropout=0.1, max_length=24):
        super().__init__()
        
        self.hidden_dim = n_feature
        self.max_length = max_length
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_length, self.hidden_dim)
        for pos in range(max_length):
            for i in range(0, self.hidden_dim, 2):
                pe[pos, i] = math.sin(pos/(10000**(2*i/self.hidden_dim)))
                if i + 1 < self.hidden_dim:
                    pe[pos, i+1] = math.cos(pos/(10000**((2*i+1)/self.hidden_dim)))
        pe = pe.unsqueeze(0)        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x*math.sqrt(self.hidden_dim)
        length = x.size(1)
        pe = Variable(self.pe[:, :length], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        x = self.dropout(x)
        
        return x

class TransformerModel(nn.Module):
    def __init__(self, d_input, n_look_back, hidden_size, n_layers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.hidden_size = hidden_size

        self.pe = PositionalEncoder(n_feature=args.n_features, dropout=dropout)

        encoder_layers = nn.TransformerEncoderLayer(d_model=d_input, nhead=d_input, dim_feedforward=hidden_size, dropout=dropout, activation='gelu')
        # encoder_layers = nn.TransformerEncoderLayer(d_model=d_input, nhead=d_input, dropout=dropout, activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.decoder = nn.Linear(d_input*n_look_back, args.n_predicted_month)

        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
        
        self.loss_fn = nn.MSELoss()
        
    
    def forward(self, X_batch, y_batch=None):
        X_batch = self.pe(X_batch)
        X_batch = self.transformer_encoder(X_batch)
        X_batch = X_batch.view(X_batch.size(0), -1)
        
        y_predicted = self.decoder(X_batch)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
        else:
            return y_predicted
        return X_batch
    
    def predict(self, X):
        with torch.no_grad():
            X = torch.tensor(X, device=args.device)
        return self.forward(X).squeeze()

# Create class to train and evaluate models
class Trainer():
    def __init__(self, city, model_type, learning_rate, important_features, train_loader, test_tensor, n_layers=2, hidden_size=128, num_filters=[100, 100, 100], dropout=0.01):
        """
        Initialise trainer, allowing input of LSTM, LSTM-ATT, or CNN 
        hyperparameters. Adam optimiser used for all models.
        """
        self.model_type = model_type    
        self.city = city    
        self.model = self.init_model(model_type, n_layers, hidden_size, num_filters, dropout, city)
        self.model.double().to(args.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.important_features, self.train_loader, self.test_tensor = important_features, train_loader, test_tensor
    
    def init_model(self, model_type, n_layers, hidden_size, num_filters, dropout, city):        
        #Get hyper param by name and set for n_layers, hidden_size, num_filters, dropout and overwrite the input param  
        # n_layers, hidden_size,num_filters, dropout= get_opt_param_by_name( algo = model_type, city = city)
    
        """Initialise a model based on whether LSTM, LSTM-ATT, or CNN is chosen."""
        if model_type.lower() == 'lstm':
            model = MultiVariateLSTM(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'lstm_attention':
            model = MultiVariateLSTM_Attention(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'cnn':
            model = MultivariateCNN(num_filters, dropout)
        elif model_type.lower() == 'transformers':
            model = TransformerModel(d_input=args.n_features, n_look_back=args.look_back, hidden_size=hidden_size, n_layers=n_layers, dropout=dropout)
        return model

    def step(self, batch):
        self.model.train()
        X_batch, y_batch = tuple(t.to(args.device) for t in batch)
        self.optimizer.zero_grad()
        y_pred, loss = self.model.forward(X_batch, y_batch)
        loss.backward()
        self.optimizer.step()
        return loss.mean().item()

    def validation(self):
        self.model.eval()
        eval_loss = 0.0

        result = {}

        y_true = np.array([])
        y_pred = np.array([])

        X_batch, y_batch = tuple(t.to(args.device) for t in self.test_tensor)
        with torch.no_grad():
            outputs, loss = self.model.forward(X_batch, y_batch)
            eval_loss = loss.mean().item()

        return eval_loss

    def train(self, epochs=20):
        # print('param of model')
        # print('hidensize',self.model.hidden_size)
        # print('nlayer',self.model.n_layers)  
        best_lost = float("inf")
        best_model = None
        for epoch in range(epochs):
            total_loss = 0.0
            for batch in self.train_loader:
                loss = self.step(batch)
                total_loss += loss
            train_loss = total_loss/len(self.train_loader)
            eval_loss = self.validation()
            if eval_loss < best_lost:
                best_lost = eval_loss
                best_model = copy.deepcopy(self.model)
            if (epoch + 1) == epochs or (epoch + 1) in [c + 1 for c in range(epochs) if c % int(epochs/4) == 0]:
                print(f"Epoch: {epoch:2}/{epochs:2} - train_loss: {train_loss:.4f} - test_loss: {eval_loss:4f}")
        self.model = best_model
        self.model.eval()
        return None



    # Lưu model vào trong thư mục models
    def save_model_to(self, path = '', city =''):       
        torch.save(self.model, path)

    def load_model_to(self, path = ''):       
        return torch.load(path)

    def evaluate_model(self, np_data=None, plot=True, scaled=True, city=None, k_steps=None, y_scaler =None):
        assert scaled, "data must be scaled"
        self.model.eval()
        tensor_data = torch.from_numpy(np_data)
        rmse_list = []
        mae_list = [] 
        mape_list = []

        y_predicted_list = []
        y_true_list = []

        for k_steps in range(1, args.n_predicted_month + 1):
            y_predicted = []
            y_true = []
            for index in range(tensor_data.size(0) - args.look_back):
                X = tensor_data[index: index + args.look_back, self.important_features]
                # yhat = self.model.predict(X.unsqueeze(0)).squeeze()

                yhat = self.model.predict(X.unsqueeze(0))
                yhat = yhat.squeeze()

                y_predicted.append(yhat.detach().cpu().numpy()[k_steps - 1])
                y_true.append(tensor_data[index + args.look_back, -1].detach().cpu().numpy())

            y_predicted = y_scaler.inverse_transform(np.array(y_predicted).reshape(-1, 1)).reshape(-1, )
            y_true = y_scaler.inverse_transform(np.array(y_true).reshape(-1, 1)).reshape(-1, )

            

            rmse = mean_squared_error(y_true, y_predicted, squared=False)
            mae = mean_absolute_error(y_true, y_predicted)
            mape = mean_absolute_percentage_error(y_true, y_predicted)

            rmse_list.append(rmse)
            mae_list.append(mae)
            mape_list.append(mape)

            print('City: '+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))          
            if plot==True:
              plt.grid(True)
              plt.plot(y_predicted, label='predicted')
              plt.plot(y_true, label='actual')
              plt.title(f"k-steps = {k_steps} - city: "+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))
              plt.legend()
              plt.show()

              plt.show()
            y_predicted_list.append(y_predicted)
            y_true_list.append(y_true)

        return y_true_list, y_predicted_list, rmse_list, mae_list, mape_list

# Get optimized Hyper param

In [11]:
def convert_str_num_filter_array(str_filter = ''):  
  arr_filter = str_filter.split(',')  
  int_arr_filter= [int(x) for x in arr_filter]
  return int_arr_filter


opt_param_cnn = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_cnn.xlsx', index_col = 'City')
opt_param_lstm = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_lstm.xlsx', index_col = 'City')
opt_param_lstm_att = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_lstm_att.xlsx', index_col = 'City')
opt_param_transformer = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_transformer.xlsx', index_col = 'City')

def get_opt_param_by_name(algo = 'transformer', city = None):
  if algo == 'cnn':
    city_param = opt_param_cnn.loc[city]
    #load  param
    n_layers = ''
    hidden_size = ''
    num_filters = convert_str_num_filter_array(city_param['Num. filters'])
    dropout = float(city_param['Dropout rate'])
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'lstm':
    city_param = opt_param_lstm.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hiden Size'])
    num_filters = ''
    dropout = ''
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'lstm_attention':
    city_param = opt_param_lstm_att.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hiden Size'])
    num_filters = ''
    dropout = ''
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'transformers':
    city_param = opt_param_transformer.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hidden Size'])
    num_filters = ''
    dropout = float(city_param['Dropout rate'])
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  return n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window


# Manual Check param before train
n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window= get_opt_param_by_name(algo='lstm', city='Lào Cai')
lookback_window,epoch, hidden_size, n_layers,learning_rate, num_filters, dropout

(3, 210, 54, 5, 0.0005838118399695072, '', '')

In [12]:
# meta_test_set_city['Lào Cai']
# meta_test_set_city['Điện Biên']

In [13]:
import requests

def send_to_telegram(message):

    apiToken = '5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q'
    chatID = '@ptn_announcement'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)

# Main Train and Predict Ensemble Meta Model

In [14]:
dt_started = datetime.now()


l_errCity = {} # có lỗi sẽ lưu vào đây, kiểm tra ngay cell sau

dict_comb_result = {}
df_all_combs_result_all = pd.DataFrame() # Lưu tất cả các loại combination
df_all_combs_2_result_all = pd.DataFrame()
df_all_combs_3_result_all = pd.DataFrame()
df_all_combs_4_result_all = pd.DataFrame()


dict_comb_metrics_result = {}
df_all_combs_metrics_result_all = pd.DataFrame() # Lưu tất cả các loại combination
df_all_combs_2_metrics_result_all = pd.DataFrame()
df_all_combs_3_metrics_result_all = pd.DataFrame()
df_all_combs_4_metrics_result_all = pd.DataFrame()

n_experiments = 2
# Loop n time and randome seed
for interation in range(n_experiments):
  seed_everything(np.int64(random.randint(1, 100)))
  for combination_features in combs_features:
    list_selected_features = [x for x in combination_features] +['Observed'] # Lấy danh sách tên các features + cột Observed 
    s_selected_features = '-'.join([str(elem) for elem in list_selected_features])
    args.n_features = len(list_selected_features) # Overwrite the global param để trả đúng n_features
    args.look_back = meta_lookback_window
    # global_epoch = 100 # no_need now

    for city in cities:
      
      try:
          # df_all_cities_result_per_com = pd.DataFrame()
          # df_all_cities_metrics_result_per_com = pd.DataFrame()

          df_one_city_result_all = pd.DataFrame()
          df_one_city_metrics_result_all = pd.DataFrame()

          origin_city_meta_test_set = meta_test_set_city[city]
          origin_city_meta_train_set = meta_train_set_city[city]     

          # Data train/test processing for all Meta model
          train = meta_train_set_city[city].astype(float)
          test = meta_test_set_city[city].astype(float)

          train = train.replace((np.inf, -np.inf, np.nan), 0)
          test = test.replace((np.inf, -np.inf, np.nan), 0) 
        
          # Đoạn này impute các trường hợp rỗng
          train = impute_missing_value(train)
          test = impute_missing_value(test)

          # Fit data scaler to training data
          full_scaler = MinMaxScaler().fit(train)
          y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

          # Scale train and test data
          train = full_scaler.transform(train)
          test = full_scaler.transform(test)


          # Pre-process data        
          ########################################################
          # Generate CNN model and define province-specific params
          ########################################################
          # Get right optimize parame
          n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = 'cnn', city = city)

          # Get data to run model
          important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, list_selected_features, origin_city_meta_test_set)    

          
          trainer = Trainer(
                              model_type='cnn',  
                              city = city,                    
                              learning_rate=learning_rate,
                              important_features=important_features,
                              train_loader=train_loader,
                              test_tensor=test_tensor,
                              num_filters= num_filters, 
                              dropout=dropout )

          # Train model with province-specific epochs
          trainer.train(epochs=epoch)           


          # Evaluate test set   
          y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)
              
          cnn_reslt = pd.DataFrame({
                                    'City': city,
                                    'Run Seq': interation,
                                    'Meta_Model_Name': 'CNN', 
                                    'Based_Models': s_selected_features,
                                    'Observed': y_true_test[0], 
                                    '1-month': y_pred_test[0], 
                                    '2-months': y_pred_test[1],
                                    '3-months': y_pred_test[2],
                                    'RMSE_1-month': rmse_list[0],
                                    'RMSE_2-month': rmse_list[1],
                                    'RMSE_3-month': rmse_list[2],
                                    'MAE_1-month': mae_list[0],
                                    'MAE_2-month': mae_list[1],
                                    'MAE_3-month': mae_list[2],
                                    'MAPE_1-month': mape_list[0],
                                    'MAPE_2-month': mape_list[1],
                                    'MAPE_3-month': mape_list[2]})
          
          cnn_metric_reslt = pd.DataFrame({
                                    'City': city,
                                    'Run Seq': interation,
                                    'Meta_Model_Name': 'CNN', 
                                    'Based_Models': s_selected_features,                                                      
                                    'RMSE_1-month': rmse_list[0],
                                    'RMSE_2-month': rmse_list[1],
                                    'RMSE_3-month': rmse_list[2],
                                    'MAE_1-month': mae_list[0],
                                    'MAE_2-month': mae_list[1],
                                    'MAE_3-month': mae_list[2],
                                    'MAPE_1-month': mape_list[0],
                                    'MAPE_2-month': mape_list[1],
                                    'MAPE_3-month': [mape_list[2]]}) # do có 1 dòng nên pandas cần index = 0    

          
          # #############################################################
          # Generate LSTM model and define province-specific params
          n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = 'lstm', city = city)
          
          # Get data to run model
          important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, list_selected_features, origin_city_meta_test_set) 
          
          trainer = Trainer(model_type='lstm', 
                            city = city,
                            learning_rate=learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            n_layers=n_layers,
                            hidden_size=hidden_size)   
        
          # Train model with province-specific epochs
          trainer.train(epochs=epoch)

          # Evaluate test set   
          y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)

          lstm_reslt = pd.DataFrame({
                                    'City': city,
                                    'Run Seq': interation,
                                    'Meta_Model_Name': 'LSTM', 
                                    'Based_Models': s_selected_features,                           
                                    'Observed': y_true_test[0], 
                                    '1-month': y_pred_test[0], 
                                    '2-months': y_pred_test[1],
                                    '3-months': y_pred_test[2],
                                    'RMSE_1-month': rmse_list[0],
                                    'RMSE_2-month': rmse_list[1],
                                    'RMSE_3-month': rmse_list[2],
                                    'MAE_1-month': mae_list[0],
                                    'MAE_2-month': mae_list[1],
                                    'MAE_3-month': mae_list[2],
                                    'MAPE_1-month': mape_list[0],
                                    'MAPE_2-month': mape_list[1],
                                    'MAPE_3-month': mape_list[2]})
          
          lstm_metric_reslt = pd.DataFrame({
                                    'City': city,
                                    'Run Seq': interation,
                                    'Meta_Model_Name': 'LSTM', 
                                    'Based_Models': s_selected_features,                                                      
                                    'RMSE_1-month': rmse_list[0],
                                    'RMSE_2-month': rmse_list[1],
                                    'RMSE_3-month': rmse_list[2],
                                    'MAE_1-month': mae_list[0],
                                    'MAE_2-month': mae_list[1],
                                    'MAE_3-month': mae_list[2],
                                    'MAPE_1-month': mape_list[0],
                                    'MAPE_2-month': mape_list[1],
                                    'MAPE_3-month': [mape_list[2]]}) # do có 1 dòng nên pandas cần index = 0

          #############################################################
          # Generate LSTM-ATT model and define province-specific params
          #############################################################
          # Get right optimize parame
          n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = 'lstm_attention', city = city)       

          
          # Get data to run model
          important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, list_selected_features, origin_city_meta_test_set)   
        
          trainer = Trainer(model_type= 'lstm_attention',
                            city = city,
                            learning_rate= learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            n_layers= n_layers,
                            hidden_size=hidden_size)

          # Train model with province-specific epochs
          trainer.train(epochs= epoch)
          
          # Evaluate test set   
          y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)   

          lstm_att_reslt = pd.DataFrame({
                                    'City': city,
                                    'Run Seq': interation,
                                    'Meta_Model_Name': 'LSTM_ATT', 
                                    'Based_Models': s_selected_features,                           
                                    'Observed': y_true_test[0], 
                                    '1-month': y_pred_test[0], 
                                    '2-months': y_pred_test[1],
                                    '3-months': y_pred_test[2],
                                    'RMSE_1-month': rmse_list[0],
                                    'RMSE_2-month': rmse_list[1],
                                    'RMSE_3-month': rmse_list[2],
                                    'MAE_1-month': mae_list[0],
                                    'MAE_2-month': mae_list[1],
                                    'MAE_3-month': mae_list[2],
                                    'MAPE_1-month': mape_list[0],
                                    'MAPE_2-month': mape_list[1],
                                    'MAPE_3-month': mape_list[2]})
          
          lstm_att_metric_reslt = pd.DataFrame({
                                    'City': city,
                                    'Run Seq': interation,
                                    'Meta_Model_Name': 'LSTM_ATT', 
                                    'Based_Models': s_selected_features,                                                      
                                    'RMSE_1-month': rmse_list[0],
                                    'RMSE_2-month': rmse_list[1],
                                    'RMSE_3-month': rmse_list[2],
                                    'MAE_1-month': mae_list[0],
                                    'MAE_2-month': mae_list[1],
                                    'MAE_3-month': mae_list[2],
                                    'MAPE_1-month': mape_list[0],
                                    'MAPE_2-month': mape_list[1],
                                    'MAPE_3-month': [mape_list[2]]}) # do có 1 dòng nên pandas cần index = 0      
          #############################################################
          # Generate Transformer model and define province-specific params
          #############################################################        
          # Get right optimize parame
          n_layers, hidden_size, _, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name(algo='transformers', city=city)
          
          # Get data to run model
          important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, list_selected_features, origin_city_meta_test_set)   
        
          # Overwrite parameters for province-specific diarrhoea prediction 
          trainer = Trainer(
                          model_type='transformers',
                          city = city, 
                          learning_rate=learning_rate,
                          important_features=important_features,
                          train_loader=train_loader,
                          test_tensor=test_tensor,
                          n_layers=n_layers,
                          hidden_size=hidden_size,
                          dropout=dropout)

          # Train model with province-specific epochs
          trainer.train(epochs=epoch)

          print("Evaluate on test set: ")
          y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)

          

          transformer_reslt = pd.DataFrame({
                                    'City': city,
                                    'Run Seq': interation,
                                    'Meta_Model_Name': 'TF', 
                                    'Based_Models': s_selected_features,
                                    'Observed': y_true_test[0], 
                                    '1-month': y_pred_test[0], 
                                    '2-months': y_pred_test[1],
                                    '3-months': y_pred_test[2],
                                    'RMSE_1-month': rmse_list[0],
                                    'RMSE_2-month': rmse_list[1],
                                    'RMSE_3-month': rmse_list[2],
                                    'MAE_1-month': mae_list[0],
                                    'MAE_2-month': mae_list[1],
                                    'MAE_3-month': mae_list[2],
                                    'MAPE_1-month': mape_list[0],
                                    'MAPE_2-month': mape_list[1],
                                    'MAPE_3-month': mape_list[2]})

          transformer_metrix_reslt = pd.DataFrame({
                                    'City': city,
                                    'Run Seq': interation,
                                    'Meta_Model_Name': 'TF', 
                                    'Based_Models': s_selected_features,                             
                                    'RMSE_1-month': rmse_list[0],
                                    'RMSE_2-month': rmse_list[1],
                                    'RMSE_3-month': rmse_list[2],
                                    'MAE_1-month': mae_list[0],
                                    'MAE_2-month': mae_list[1],
                                    'MAE_3-month': mae_list[2],
                                    'MAPE_1-month': mape_list[0],
                                    'MAPE_2-month': mape_list[1],
                                    'MAPE_3-month': [mape_list[2]]}) # do có 1 dòng nên pandas cần index = 0

          # Lưu kết quả  predict 36 months
          df_one_city_result_all = pd.concat([df_one_city_result_all, cnn_reslt,lstm_reslt,lstm_att_reslt,transformer_reslt], axis=0)
          # df_one_city_result_all = df_one_city_result_all.append(cnn_reslt)
          # df_one_city_result_all = df_one_city_result_all.append(lstm_reslt)
          # df_one_city_result_all = df_one_city_result_all.append(lstm_att_reslt)
          # df_one_city_result_all = df_one_city_result_all.append(transformer_reslt)

          # Lưu kết quả metrics
          df_one_city_metrics_result_all = pd.concat([df_one_city_metrics_result_all, cnn_metric_reslt,
                                      lstm_metric_reslt,lstm_att_metric_reslt,transformer_metrix_reslt], axis=0)
          # df_one_city_metrics_result_all = df_one_city_metrics_result_all.append(cnn_metric_reslt) 
          # df_one_city_metrics_result_all = df_one_city_metrics_result_all.append(lstm_metric_reslt)
          # df_one_city_metrics_result_all = df_one_city_metrics_result_all.append(lstm_att_metric_reslt)
          # df_one_city_metrics_result_all = df_one_city_metrics_result_all.append(transformer_metrix_reslt)

     
          # Lưu DF theo từng loại combination
          df_all_combs_result_all = pd.concat([df_all_combs_result_all, df_one_city_result_all], axis=0)
          df_all_combs_metrics_result_all = pd.concat([df_all_combs_metrics_result_all, df_one_city_metrics_result_all], axis=0)
          # df_all_combs_result_all = df_all_combs_result_all.append(df_one_city_result_all)
          # df_all_combs_metrics_result_all = df_all_combs_metrics_result_all.append(df_one_city_metrics_result_all)  

          if(len(combination_features) == 2):
            df_all_combs_2_result_all = pd.concat([df_all_combs_2_result_all, df_one_city_result_all], axis=0)
            df_all_combs_2_metrics_result_all = pd.concat([df_all_combs_2_metrics_result_all, df_one_city_metrics_result_all], axis=0)
            # df_all_combs_2_result_all = df_all_combs_2_result_all.append(df_one_city_result_all)
            # df_all_combs_2_metrics_result_all = df_all_combs_2_metrics_result_all.append(df_one_city_metrics_result_all)
          if(len(combination_features) == 3):
            df_all_combs_3_result_all = pd.concat([df_all_combs_3_result_all, df_one_city_result_all], axis=0)
            df_all_combs_3_metrics_result_all = pd.concat([df_all_combs_3_metrics_result_all, df_one_city_metrics_result_all], axis=0)
            # df_all_combs_3_result_all = df_all_combs_3_result_all.append(df_one_city_result_all)
            # df_all_combs_3_metrics_result_all = df_all_combs_3_metrics_result_all.append(df_one_city_metrics_result_all)
          if(len(combination_features) == 4):
            df_all_combs_4_result_all = pd.concat([df_all_combs_4_result_all, df_one_city_result_all], axis=0)
            df_all_combs_4_metrics_result_all = pd.concat([df_all_combs_4_metrics_result_all, df_one_city_metrics_result_all], axis=0)
            # df_all_combs_4_result_all = df_all_combs_4_result_all.append(df_one_city_result_all)
            # df_all_combs_4_metrics_result_all = df_all_combs_4_metrics_result_all.append(df_one_city_metrics_result_all)

          # lưu xuống nhiều lần, tránh mất dữ liệu khi train.
          df_all_combs_result_all.to_excel(com_path)
          df_all_combs_2_result_all.to_excel(com_path_2)
          df_all_combs_3_result_all.to_excel(com_path_3)
          df_all_combs_4_result_all.to_excel(com_path_4)

          df_all_combs_metrics_result_all.to_excel(com_path_metrics)
          df_all_combs_2_metrics_result_all.to_excel(com_path_metrics_2)
          df_all_combs_3_metrics_result_all.to_excel(com_path_metrics_3)
          df_all_combs_4_metrics_result_all.to_excel(com_path_metrics_4)

#           send_to_telegram("Server Chạy Xong validation Simple " + str(city))
          
        
      except Exception as e:
        l_errCity[city] = e
        break    
  

# dt_ended = datetime.now()
# run_time = round((dt_ended - dt_started).total_seconds()/60)
# print('kết thúc train trong:', run_time)
# print ('danh sách error city: ')
# print(l_errCity)

# send_to_telegram("Server Chạy Xong validation Simple")


<ipython-input-10-538c6d9183a6>:3: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seed)


Epoch:  0/490 - train_loss: 0.8237 - test_loss: 0.620199
Epoch: 122/490 - train_loss: 0.1753 - test_loss: 0.160211
Epoch: 244/490 - train_loss: 0.1738 - test_loss: 0.153849
Epoch: 366/490 - train_loss: 0.1745 - test_loss: 0.154263
Epoch: 488/490 - train_loss: 0.1748 - test_loss: 0.160979
Epoch: 489/490 - train_loss: 0.1785 - test_loss: 0.154969
City: Thanh Hóa  _algo:cnn  -RMSE: 8.673861037088708
City: Thanh Hóa  _algo:cnn  -RMSE: 8.747877502340542
City: Thanh Hóa  _algo:cnn  -RMSE: 10.147840285636754
Epoch:  0/500 - train_loss: 0.7689 - test_loss: 0.652916
Epoch: 125/500 - train_loss: 0.1351 - test_loss: 0.143580
Epoch: 250/500 - train_loss: 0.0960 - test_loss: 0.143363
Epoch: 375/500 - train_loss: 0.0621 - test_loss: 0.148980
Epoch: 499/500 - train_loss: 0.0207 - test_loss: 0.214917
City: Thanh Hóa  _algo:lstm  -RMSE: 6.868907717107989
City: Thanh Hóa  _algo:lstm  -RMSE: 6.835533484597226
City: Thanh Hóa  _algo:lstm  -RMSE: 6.82754932538689
Epoch:  0/420 - train_loss: 0.8092 - test_l

Epoch: 199/200 - train_loss: 0.0979 - test_loss: 0.049410
City: Trà Vinh  _algo:cnn  -RMSE: 11.272544180504154
City: Trà Vinh  _algo:cnn  -RMSE: 10.959694953774557
City: Trà Vinh  _algo:cnn  -RMSE: 15.698001215554399
Epoch:  0/380 - train_loss: 0.6952 - test_loss: 0.640461
Epoch: 95/380 - train_loss: 0.1056 - test_loss: 0.065364
Epoch: 190/380 - train_loss: 0.0804 - test_loss: 0.046876
Epoch: 285/380 - train_loss: 0.0778 - test_loss: 0.044846
Epoch: 379/380 - train_loss: 0.0867 - test_loss: 0.037830
City: Trà Vinh  _algo:lstm  -RMSE: 8.865611442946618
City: Trà Vinh  _algo:lstm  -RMSE: 8.079603746387987
City: Trà Vinh  _algo:lstm  -RMSE: 12.666134567330268
Epoch:  0/370 - train_loss: 0.7301 - test_loss: 0.577121
Epoch: 92/370 - train_loss: 0.1339 - test_loss: 0.068770
Epoch: 184/370 - train_loss: 0.0862 - test_loss: 0.054235
Epoch: 276/370 - train_loss: 0.0771 - test_loss: 0.044049
Epoch: 368/370 - train_loss: 0.0783 - test_loss: 0.074871
Epoch: 369/370 - train_loss: 0.0790 - test_loss

Epoch: 190/380 - train_loss: 0.1461 - test_loss: 0.077419
Epoch: 285/380 - train_loss: 0.1341 - test_loss: 0.079559
Epoch: 379/380 - train_loss: 0.0896 - test_loss: 0.105609
City: Yên Bái  _algo:lstm  -RMSE: 11.881219870935688
City: Yên Bái  _algo:lstm  -RMSE: 12.86006677124127
City: Yên Bái  _algo:lstm  -RMSE: 12.110723358137891
Epoch:  0/440 - train_loss: 0.9794 - test_loss: 0.792956
Epoch: 110/440 - train_loss: 0.2034 - test_loss: 0.142668
Epoch: 220/440 - train_loss: 0.1842 - test_loss: 0.102942
Epoch: 330/440 - train_loss: 0.1820 - test_loss: 0.102098
Epoch: 439/440 - train_loss: 0.1756 - test_loss: 0.101980
City: Yên Bái  _algo:lstm_attention  -RMSE: 8.78024351331572
City: Yên Bái  _algo:lstm_attention  -RMSE: 13.176919265462644
City: Yên Bái  _algo:lstm_attention  -RMSE: 16.884379020273162
Epoch:  0/430 - train_loss: 1.0503 - test_loss: 0.782955
Epoch: 107/430 - train_loss: 0.3017 - test_loss: 0.360128
Epoch: 214/430 - train_loss: 0.2975 - test_loss: 0.370665
Epoch: 321/430 - tr

Epoch:  0/190 - train_loss: 0.6931 - test_loss: 0.624048
Epoch: 47/190 - train_loss: 0.1403 - test_loss: 0.083273
Epoch: 94/190 - train_loss: 0.1283 - test_loss: 0.062238
Epoch: 141/190 - train_loss: 0.1279 - test_loss: 0.060160
Epoch: 188/190 - train_loss: 0.1282 - test_loss: 0.062094
Epoch: 189/190 - train_loss: 0.1266 - test_loss: 0.063944
City: Đắk Lắk  _algo:lstm_attention  -RMSE: 13.339628895295798
City: Đắk Lắk  _algo:lstm_attention  -RMSE: 11.137658949601791
City: Đắk Lắk  _algo:lstm_attention  -RMSE: 11.671316261433839
Epoch:  0/220 - train_loss: 1.5395 - test_loss: 1.595927
Epoch: 55/220 - train_loss: 0.1668 - test_loss: 0.089078
Epoch: 110/220 - train_loss: 0.1617 - test_loss: 0.085318
Epoch: 165/220 - train_loss: 0.1629 - test_loss: 0.091246
Epoch: 219/220 - train_loss: 0.1588 - test_loss: 0.085912
Evaluate on test set: 
City: Đắk Lắk  _algo:transformers  -RMSE: 15.625509342671636
City: Đắk Lắk  _algo:transformers  -RMSE: 18.29364780778236
City: Đắk Lắk  _algo:transformers 

...

Epoch: 150/200 - train_loss: 0.1544 - test_loss: 0.087160
Epoch: 199/200 - train_loss: 0.1335 - test_loss: 0.078731
Evaluate on test set: 
City: Vĩnh Phúc  _algo:transformers  -RMSE: 16.04455664993921
City: Vĩnh Phúc  _algo:transformers  -RMSE: 14.392087186913729
City: Vĩnh Phúc  _algo:transformers  -RMSE: 23.56296445713315
Epoch:  0/390 - train_loss: 0.8192 - test_loss: 0.709321
Epoch: 97/390 - train_loss: 0.1862 - test_loss: 0.107204
Epoch: 194/390 - train_loss: 0.1682 - test_loss: 0.106027
Epoch: 291/390 - train_loss: 0.1592 - test_loss: 0.097404
Epoch: 388/390 - train_loss: 0.1741 - test_loss: 0.113996
Epoch: 389/390 - train_loss: 0.1606 - test_loss: 0.122981
City: Yên Bái  _algo:cnn  -RMSE: 8.23264559783523
City: Yên Bái  _algo:cnn  -RMSE: 9.362409803816881
City: Yên Bái  _algo:cnn  -RMSE: 13.461411752674493
Epoch:  0/380 - train_loss: 0.7772 - test_loss: 0.796147
Epoch: 95/380 - train_loss: 0.1708 - test_loss: 0.112025
Epoch: 190/380 - train_loss: 0.1468 - test_loss: 0.107880
Epo

Epoch: 37/150 - train_loss: 0.3559 - test_loss: 0.332033
Epoch: 74/150 - train_loss: 0.1904 - test_loss: 0.145643
Epoch: 111/150 - train_loss: 0.1457 - test_loss: 0.077502
Epoch: 148/150 - train_loss: 0.1261 - test_loss: 0.065363
Epoch: 149/150 - train_loss: 0.1227 - test_loss: 0.064572
City: Đắk Lắk  _algo:cnn  -RMSE: 9.971933248911894
City: Đắk Lắk  _algo:cnn  -RMSE: 11.285023890427693
City: Đắk Lắk  _algo:cnn  -RMSE: 11.340218878044675
Epoch:  0/380 - train_loss: 0.6626 - test_loss: 0.578820
Epoch: 95/380 - train_loss: 0.1257 - test_loss: 0.067427
Epoch: 190/380 - train_loss: 0.1102 - test_loss: 0.081544
Epoch: 285/380 - train_loss: 0.1246 - test_loss: 0.061306
Epoch: 379/380 - train_loss: 0.1236 - test_loss: 0.071913
City: Đắk Lắk  _algo:lstm  -RMSE: 16.164256426008695
City: Đắk Lắk  _algo:lstm  -RMSE: 15.128072612596961
City: Đắk Lắk  _algo:lstm  -RMSE: 13.03277695576233
Epoch:  0/190 - train_loss: 0.6900 - test_loss: 0.623395
Epoch: 47/190 - train_loss: 0.1343 - test_loss: 0.0792

Epoch: 57/230 - train_loss: 0.1002 - test_loss: 0.202199
Epoch: 114/230 - train_loss: 0.0565 - test_loss: 0.251486
Epoch: 171/230 - train_loss: 0.0413 - test_loss: 0.325786
Epoch: 228/230 - train_loss: 0.0474 - test_loss: 0.277386
Epoch: 229/230 - train_loss: 0.0477 - test_loss: 0.272907
City: Thái Nguyên  _algo:lstm  -RMSE: 31.469654148753655
City: Thái Nguyên  _algo:lstm  -RMSE: 31.636561816545157
City: Thái Nguyên  _algo:lstm  -RMSE: 33.19852665672224
Epoch:  0/230 - train_loss: 0.6375 - test_loss: 0.547392
Epoch: 57/230 - train_loss: 0.1121 - test_loss: 0.202558
Epoch: 114/230 - train_loss: 0.0841 - test_loss: 0.214148
Epoch: 171/230 - train_loss: 0.1054 - test_loss: 0.196486
Epoch: 228/230 - train_loss: 0.1145 - test_loss: 0.191591
Epoch: 229/230 - train_loss: 0.1353 - test_loss: 0.187293
City: Thái Nguyên  _algo:lstm_attention  -RMSE: 27.486694147351095
City: Thái Nguyên  _algo:lstm_attention  -RMSE: 30.58902155475403
City: Thái Nguyên  _algo:lstm_attention  -RMSE: 29.77134384918

Epoch: 122/490 - train_loss: 0.1635 - test_loss: 0.110053
Epoch: 244/490 - train_loss: 0.1394 - test_loss: 0.083747
Epoch: 366/490 - train_loss: 0.1197 - test_loss: 0.078165
Epoch: 488/490 - train_loss: 0.0997 - test_loss: 0.078189
Epoch: 489/490 - train_loss: 0.0989 - test_loss: 0.074349
City: Tây Ninh  _algo:lstm_attention  -RMSE: 6.501319590656525
City: Tây Ninh  _algo:lstm_attention  -RMSE: 6.302419590325466
City: Tây Ninh  _algo:lstm_attention  -RMSE: 5.19733825885406
Epoch:  0/200 - train_loss: 0.9441 - test_loss: 0.644100
Epoch: 50/200 - train_loss: 0.3180 - test_loss: 0.276362
Epoch: 100/200 - train_loss: 0.3197 - test_loss: 0.268895
Epoch: 150/200 - train_loss: 0.3217 - test_loss: 0.301148
Epoch: 199/200 - train_loss: 0.3158 - test_loss: 0.265222
Evaluate on test set: 
City: Tây Ninh  _algo:transformers  -RMSE: 17.557716430010615
City: Tây Ninh  _algo:transformers  -RMSE: 19.92681169672377
City: Tây Ninh  _algo:transformers  -RMSE: 19.34678715218079
Epoch:  0/420 - train_loss:

Epoch: 92/370 - train_loss: 0.1259 - test_loss: 0.060063
Epoch: 184/370 - train_loss: 0.1251 - test_loss: 0.054224
Epoch: 276/370 - train_loss: 0.1302 - test_loss: 0.057379
Epoch: 368/370 - train_loss: 0.1255 - test_loss: 0.060342
Epoch: 369/370 - train_loss: 0.1250 - test_loss: 0.059583
Evaluate on test set: 
City: Đà Nẵng  _algo:transformers  -RMSE: 29.787059821747718
City: Đà Nẵng  _algo:transformers  -RMSE: 19.07304437798737
City: Đà Nẵng  _algo:transformers  -RMSE: 25.104853471359128
Epoch:  0/170 - train_loss: 0.8891 - test_loss: 0.773028
Epoch: 42/170 - train_loss: 0.6758 - test_loss: 0.617150
Epoch: 84/170 - train_loss: 0.5827 - test_loss: 0.545037
Epoch: 126/170 - train_loss: 0.5177 - test_loss: 0.478084
Epoch: 168/170 - train_loss: 0.4517 - test_loss: 0.416019
Epoch: 169/170 - train_loss: 0.4526 - test_loss: 0.414657
City: Đắk Nông  _algo:cnn  -RMSE: 10.508510768764149
City: Đắk Nông  _algo:cnn  -RMSE: 9.749861090814154
City: Đắk Nông  _algo:cnn  -RMSE: 8.797074862994059
Epoc

In [15]:
run_time = round((dt_ended - dt_started).total_seconds()/60)
print('kết thúc train trong:', run_time)
print ('danh sách error city: ')
print(l_errCity)

NameError: name 'dt_ended' is not defined

In [ ]:
# import requests

# def send_to_telegram(message):

#     apiToken = '5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q'
#     chatID = '@ptn_announcement'
#     apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

#     try:
#         response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
#         print(response.text)
#     except Exception as e:
#         print(e)

# send_to_telegram("Server Chạy Xong\nList city:\n" + ',\n'.join(str(city) for city in cities) + "\nalgo: ensemble" )

# final result

# Đọc kết quả 1997-2015 của 44 combination. 

In [ ]:
prj_path = 'H:/Store/Research/UIT/PhD Research/Dev Environment/Tropical Diseases/running/'
path_simple_result_2015 =      prj_path+ 'results/ensemble/result/simple_combination_4_model_2015/all_combinations_metrics_1.xlsx'

df_result_simple_2015 = pd.read_excel(path_simple_result_2015)

# df_result_simple_2015 = df_result_simple_2015.loc[:,["City","Meta_Model_Name","Based_Models",'RMSE_1-month']
df_result_simple_2015 = df_result_simple_2015.loc[:,["City","Meta_Model_Name","Based_Models",'MAE_1-month']]

df_result_simple_2015['Model Name'] = df_result_simple_2015['Meta_Model_Name']+'*'+ df_result_simple_2015['Based_Models']
# df_result_simple_2015 = df_result_simple_2015.loc[:,["City","RMSE_1-month","Model Name"]]
df_result_simple_2015 = df_result_simple_2015.loc[:,["City","MAE_1-month","Model Name"]]
df_result_simple_2015

# Xếp hạng theo rank

In [ ]:
pivot_simple = df_result_simple_2015.pivot(index='City', columns='Model Name', values='RMSE_1-month').reset_index(drop=True)
# pivot_simple = pivot_simple.reset_index()
ranked_pivot_simple = pivot_simple.rank(axis=1)
ranked_pivot_simple.head()
ranked_pivot_simple.to_excel('Ranked_44_2015.xlsx')


In [ ]:
ranked_pivot_simple = pd.read_excel('Ranked_44_2015.xlsx')
ranked_pivot_simple = ranked_pivot_simple.drop(columns=['Unnamed: 0'])
ranked_pivot_simple

In [ ]:
ranked_pivot_simple.mean(axis=0).sort_values()

In [ ]:
tp_ranked_pivot_simple = ranked_pivot_simple.transpose()
# tp_ranked_pivot_simple.reset_index("Model Name", inplace = True)
tp_ranked_pivot_simple['Mean_Rank']= tp_ranked_pivot_simple.mean(axis=1)
rank_sort_simple = tp_ranked_pivot_simple.sort_values(by = 'Mean_Rank', ascending=True)
top_rank_simple = rank_sort_simple.head(5)
top_rank_simple


LSTM_ATT*cnn_pred-lstm_att_pred-lstm_pred-Observed            15.945455
LSTM_ATT*lstm_pred-tf_pred-Observed                           16.127273
LSTM*cnn_pred-lstm_att_pred-tf_pred-Observed                  16.509091
LSTM*cnn_pred-lstm_pred-Observed                              16.654545
LSTM*lstm_att_pred-tf_pred-Observed                           16.854545

In [ ]:
# Đoạn này dùng cho theo rank
top_n_of_simple_2015 = pd.DataFrame(data= ['LSTM_ATT*cnn_pred-lstm_att_pred-lstm_pred-Observed',
                                           'LSTM_ATT*lstm_pred-tf_pred-Observed',
                                           'LSTM*cnn_pred-lstm_att_pred-tf_pred-Observed',                                           
                                            'LSTM*cnn_pred-lstm_pred-Observed',
                                            'LSTM*lstm_att_pred-tf_pred-Observed'], columns=['Model Name'])
top_n_of_simple_2015

# Xếp hạng 44 bộ kết hợp cho kết quả 1997-2015 theo rmse

In [ ]:
# Get n_top theo rmse cố định cho tất cả thành phố
def get_n_top_in(df_combination ='', n_best =3):
    model_group_mean = df_combination.groupby(['Model Name'],as_index = False).mean()
    return model_group_mean.sort_values(by='RMSE_1-month',ascending=True).head(n_best).reset_index(drop=True)
top_n_of_simple_2015 = get_n_top_in (df_combination= df_result_simple_2015, n_best =5 )
top_n_of_simple_2015

# Theo MAE

In [ ]:
# Get n_top theo rmse cố định cho tất cả thành phố
def get_n_top_in(df_combination ='', n_best =3):
    model_group_mean = df_combination.groupby(['Model Name'],as_index = False).mean()
    return model_group_mean.sort_values(by='MAE_1-month',ascending=True).head(n_best).reset_index(drop=True)
top_n_of_simple_2015 = get_n_top_in (df_combination= df_result_simple_2015, n_best =5 )
top_n_of_simple_2015

In [ ]:
top_n_of_simple_2015.to_excel('list.xlsx')

In [ ]:
# Get n_top theo rmse theo mỗi thành phố (linh động)
def get_n_top_each_city_in(df_combination ='', n_best =5):
    df = df_result_simple_2015.sort_values(by = ['RMSE_1-month'], ascending=True).groupby('City').head(n_best).reset_index()
    return df
top_n_of_simple_2015 = get_n_top_each_city_in (df_combination= df_result_simple_2015, n_best = 5 )
top_n_of_simple_2015

In [ ]:
# Get n_top theo rmse theo mỗi thành phố (linh động)
def get_n_top_each_city_in(df_combination ='', n_best =5):
    df = df_result_simple_2015.sort_values(by = ['MAE_1-month'], ascending=True).groupby('City').head(n_best).reset_index()
    return df
top_n_of_simple_2015 = get_n_top_each_city_in (df_combination= df_result_simple_2015, n_best = 5 )
top_n_of_simple_2015

# Set cứng tính sau cho bộ rank này
LSTM_ATT*cnn_pred-lstm_att_pred-lstm_pred-Observed
LSTM_ATT*lstm_pred-tf_pred-Observed
LSTM*cnn_pred-lstm_att_pred-tf_pred-Observed
LSTM*cnn_pred-lstm_pred-Observed
LSTM*lstm_att_pred-tf_pred-Observed

CNN*lstm_att_pred-tf_pred-Observed

In [ ]:
# Bộ này tự tạo
top_n_of_simple_2015 = pd.DataFrame(data= ['CNN*lstm_att_pred-tf_pred-Observed','CNN*cnn_pred-lstm_pred-Observed','CNN*lstm_pred-tf_pred-Observed'], columns=['Model Name'])
top_n_of_simple_2015


# Đọc kết quả Predict của 44 combination 1997-2018

In [ ]:
prj_path_result_simple_ensemple_2018 = 'H:/Store/Research/UIT/PhD Research/Dev Environment/Tropical Diseases/running/eval_result/\simple_combination_14_12/'

df_simple_ensemble_2018 = pd.read_excel(prj_path_result_simple_ensemple_2018+'all_combinations_1.xlsx')
df_simple_ensemble_2018 = df_simple_ensemble_2018.loc[:,["City","Meta_Model_Name","Based_Models",'Observed','1-month','RMSE_1-month','MAE_1-month','MAE_1-month']]
# Không sort và lấy best result trong mỗi city. vì filter dùng df_result_simple_2015
df_simple_ensemble_2018['Model Name'] = df_simple_ensemble_2018['Meta_Model_Name']+'*'+ df_simple_ensemble_2018['Based_Models']
df_simple_ensemble_2018

# Convert kết quả 2015 thành 2018

In [ ]:
# Convert Cứng n giải thuật cho cả kiểu rank và rmse
convert_simple_2015_to_2018 = df_simple_ensemble_2018.merge(top_n_of_simple_2015[['Model Name']], on=['Model Name'])
short_convert_simple_2015_to_2018 = convert_simple_2015_to_2018.loc[:,['City','Observed','1-month','Model Name']]
short_convert_simple_2015_to_2018.head()

In [ ]:
# Conver linh động cho mỗi thành phố n giải thuật khác nhau
convert_simple_2015_to_2018 = df_simple_ensemble_2018.merge(top_n_of_simple_2015[['Model Name','City']], on=['Model Name','City'])
short_convert_simple_2015_to_2018 = convert_simple_2015_to_2018.loc[:,['City','Observed','1-month','Model Name']]
short_convert_simple_2015_to_2018.head()

In [ ]:
# convert_simple_2015_to_2018.to_excel('convert_simple_2015_to_2018.xlsx')

In [ ]:
# test = short_convert_simple_2015_to_2018.loc[(short_convert_simple_2015_to_2018['City']=='An Giang') & 
#                                             (short_convert_simple_2015_to_2018['Model Name']=='LSTM*cnn_pred-lstm_pred-Observed')]
# test.head()

In [ ]:
# top_n_of_simple_2015['Model Name'].to_numpy()

In [ ]:
# for model_name in top_n_of_simple_2015['Model Name'].to_numpy():
#         print(model_name)

# Tạo bộ kết quả trung bình cho mỗi thành phố. Cho dạng linh động mỗi thành phố N combine khác nhau

In [ ]:
# Đoạn này hiện đang tính simple combination 5 bộ giải thuật khác nhau mỗi thành phố
cities = ['An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
        'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
        'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
        'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
        'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
        'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
        'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
        'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
         'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp']
dict_all_cities_simple = {}
df_mix_all = pd.DataFrame()
for city in cities:
        df_one_city = pd.DataFrame()        
        for model_name in top_n_of_simple_2015.loc[top_n_of_simple_2015['City']==city]['Model Name'].to_numpy():
                selected_result = short_convert_simple_2015_to_2018.loc[(short_convert_simple_2015_to_2018['City']==city) & 
                                            (short_convert_simple_2015_to_2018['Model Name']== model_name)]
                selected_result = selected_result.reset_index(drop=True)  
                selected_result = selected_result.rename(columns={"1-month": model_name})
                df_one_city = pd.concat([df_one_city,selected_result[model_name]], axis=1)
        
        df_one_city['Simple Average'] = df_one_city.mean(axis=1)
        df_one_city['Observed'] = selected_result['Observed']
        df_one_city['RMSE_Simple_Avg'] = mean_squared_error(df_one_city['Observed'], df_one_city['Simple Average'], squared=False)
        df_one_city['MAE_Simple_Avg'] = mean_absolute_error(df_one_city['Observed'], df_one_city['Simple Average'])
        df_one_city['City'] = city
        dict_all_cities_simple[city] = df_one_city
        df_mix_all = pd.concat([df_mix_all,df_one_city])

df_simple_mix_all_metrix= df_mix_all.groupby(['City']).head(1)
df_simple_mix_all_metrix = df_simple_mix_all_metrix.reset_index(drop=True)
df_simple_mix_all_metrix = df_simple_mix_all_metrix[['City','RMSE_Simple_Avg','MAE_Simple_Avg']]
df_simple_mix_all_metrix['Alg_name'] = 'Simple Ensemble'
df_simple_mix_all_metrix.head()

# Tạo ra Bộ kết quả Simple combination = trung bình của 5 combination
Cho dạng cứng 5 combi giống nhau cho tất cả thành phố

In [ ]:
# Đoạn này hiện đang tính simple combination cứng cho 5 bộ giải thuật
cities = ['An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
        'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
        'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
        'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
        'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
        'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
        'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
        'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
         'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp']
dict_all_cities_simple = {}
df_mix_all = pd.DataFrame()
for city in cities:
        df_one_city = pd.DataFrame()        
        for model_name in top_n_of_simple_2015['Model Name'].to_numpy():
                selected_result = short_convert_simple_2015_to_2018.loc[(short_convert_simple_2015_to_2018['City']==city) & 
                                            (short_convert_simple_2015_to_2018['Model Name']== model_name)]
                selected_result = selected_result.reset_index(drop=True)  
                selected_result = selected_result.rename(columns={"1-month": model_name})
                df_one_city = pd.concat([df_one_city,selected_result[model_name]], axis=1)
        df_one_city['Simple Average'] = df_one_city.mean(axis=1)
        df_one_city['Observed'] = selected_result['Observed']
        df_one_city['RMSE_Simple_Avg'] = mean_squared_error(df_one_city['Observed'], df_one_city['Simple Average'], squared=False)
        df_one_city['MAE_Simple_Avg'] = mean_absolute_error(df_one_city['Observed'], df_one_city['Simple Average'])
        df_one_city['City'] = city
        dict_all_cities_simple[city] = df_one_city
        df_mix_all = pd.concat([df_mix_all,df_one_city])

df_simple_mix_all_metrix= df_mix_all.groupby(['City']).head(1)
df_simple_mix_all_metrix = df_simple_mix_all_metrix.reset_index(drop=True)
df_simple_mix_all_metrix = df_simple_mix_all_metrix[['City','RMSE_Simple_Avg','MAE_Simple_Avg']]
df_simple_mix_all_metrix['Alg_name'] = 'Simple Ensemble'
df_simple_mix_all_metrix.head()


In [ ]:
df_simple_mix_all_metrix.rename(columns = {'RMSE_Simple_Avg':'RMSE_1-month','MAE_Simple_Avg':'MAE_1-month'}, inplace = True)
df_simple_mix_all_metrix.to_excel('simple_ensemble_all_mae.xlsx')

# Đọc kết quả Deep Learning, Traditional... Và so sánh với simple

In [ ]:
prj_path = 'H:/Store/Research/UIT/PhD Research/Dev Environment/Tropical Diseases/running'

result_path = 'H:/Store/Research/UIT/PhD Research/Dev Environment/Tropical Diseases/running/eval_result/MLJ 12_01/'
result_path_3algo = result_path + "cnn_lstm_lstm-att_30_11_2022/"
result_path_tf = result_path + "transformer_08_12_2022/"
result_path_sarima = result_path + "sarima_08_12_2022/"
result_path_sarimax = result_path + "sarimax_08_12_2022/"
result_path_poisson = result_path + "poisson_10_01_2023/"
result_path_xgboost = result_path + "xgboost_10_01_2023/"
result_path_svmrbf = result_path + "svmrbf_08_12_2022/"
result_path_ensemble = result_path + "ensemble_14_12_2022/result\weighted_combination/"
result_path_weighted_ensemble = result_path + "weighted_combination_14_12/"
result_path_dart = result_path + "dart_01_12/"
cities = ['An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
        'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
        'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
        'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
        'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
        'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
        'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
        'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
         'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp']

# cities = ['Điện Biên', 'Thái Bình', 'Lào Cai', 'Kon Tum', 'Cao Bằng','Lâm Đồng']
z=55

# Lấy dạng Dataframe 3 kết quả metrix của tất cả thành phố theo eval_metrix
def get_df_best_metrix_results_by(eval_metrix='MAE_1-month', result_algo_path='', file_head='', file_tail='_0'):
    # Lọc ra theo min RMSE - của cả 3 giải thuật
    l_all_algo_cities = pd.DataFrame()
    for city in cities:
        city_result = pd.read_excel(
            result_algo_path + file_head + city + file_tail + '.xlsx')
        city_result = city_result[['City', 'Alg_name', 'RMSE_1-month','MAE_1-month']]
        # group by Alg_name, sort RMSE_1-month, select first row for each group
        selected_result = city_result.sort_values(
            by=[eval_metrix], ascending=True).groupby('Alg_name').head(1)
        selected_result = selected_result.sort_index(axis=0, ascending=True)

        # lấy kết quả theo giải thuật
        if len(l_all_algo_cities) == 0:
            l_all_algo_cities = selected_result
        else: 
            l_all_algo_cities = pd.concat([l_all_algo_cities, selected_result])
    return l_all_algo_cities

# Lọc ra theo từng giải thuật ứng với từng city
def extract_by_algo(algo='cnn', city_data=''):
    result = city_data.loc[city_data['Alg_name'] == algo]
    # result['Steps']= range(0,36)
    result.insert(0, 'Months', range(0, len(result)))
    # result = result.drop(['Steps'], axis=1)
    result = result.set_index('Months')
    return result
def get_dict_result(dict):
    lstm_att_result = {}
    cnn_result = {}
    lstm_result = {}
    tf_result = {}
    sarima_result = {}
    ensemble_result = {}
    sarimax_result = {}
    svm_result = {}
    xgboost_result = {}
    ps_result = {}
    for city in cities:
        one_city = dict[city]

        ps_one_city = extract_by_algo(algo='poisson', city_data=one_city)
        ps_result[city] = ps_one_city

        xgboost_one_city = extract_by_algo(algo='xgboost', city_data=one_city)
        xgboost_result[city] = xgboost_one_city

        svm_one_city = extract_by_algo(algo='SVM-BRF', city_data=one_city)
        svm_result[city] = svm_one_city

        sarima_one_city = extract_by_algo(algo='SARIMA', city_data=one_city)
        sarima_result[city] = sarima_one_city

        ensemble_one_city = extract_by_algo(
            algo='ensemble', city_data=one_city)
        ensemble_result[city] = ensemble_one_city

        sarimax_one_city = extract_by_algo(algo='SARIMAX', city_data=one_city)
        sarimax_result[city] = sarimax_one_city

        cnn_one_city = extract_by_algo(algo='cnn', city_data=one_city)
        cnn_result[city] = cnn_one_city

        lstm_one_city = extract_by_algo(algo='lstm', city_data=one_city)
        lstm_result[city] = lstm_one_city

        lstm_att_one_city = extract_by_algo(
            algo='lstm_attention', city_data=one_city)
        lstm_att_result[city] = lstm_att_one_city

        tf_one_city = extract_by_algo(algo='transformers', city_data=one_city)
        tf_result[city] = tf_one_city
    # return   svm_result , xgboost_result, ps_result , sarima_result, sarimax_result
    return lstm_att_result, cnn_result, lstm_result, tf_result, sarima_result, ensemble_result, svm_result, xgboost_result, ps_result, sarimax_result
def process_dart():
    dart_model_dict = {

    }
    model_list = ['BlockRNNModel', 'CatBoostModel', 'LightGBMModel', 'LinearRegressionModel',
                  'NHiTSModel', 'RandomForest', 'TCNModel', 'TFTModel', 'TransformerModel']
    BlockRNNModel = pd.DataFrame()

    for model in model_list:
        cities_result = pd.DataFrame()
        for city in cities:
            city_result = pd.read_excel(
                result_path_dart + model + "/" + "fb_" + city + ".xlsx")
            city_result['Alg_name'] = model
            city_result = city_result[['City', 'Alg_name', 'RMSE_1-month','MAE_1-month']]
            if len(cities_result) == 0:
                cities_result = city_result
            else:
                cities_result = pd.concat([cities_result, city_result])
        dart_model_dict[model] = cities_result
    return dart_model_dict

result_3algo = get_df_best_metrix_results_by(result_algo_path=result_path_3algo, file_head='diarrhoea_')
result_cnn= extract_by_algo(algo='cnn', city_data =result_3algo)
result_lstm = extract_by_algo(algo='lstm', city_data =result_3algo)
result_lstm_att = extract_by_algo(algo='lstm_attention', city_data =result_3algo)

# tf
result_tf = get_df_best_metrix_results_by(result_algo_path=result_path_tf, file_head="diarrhoea_transformer_")

# sarima
result_sarima = get_df_best_metrix_results_by(result_algo_path=result_path_sarima, file_head="diarrhoea_sarima_")

# sarimax
result_sarimax = get_df_best_metrix_results_by(result_algo_path=result_path_sarimax, file_head="diarrhoea_sarimax_")


In [ ]:
simple_ensemble = df_simple_mix_all_metrix
simple_ensemble.to_excel('simple_ensemble_all_mae.xlsx')

# Đọc nhanh kết quả simple ensemble theo MAE của top 5

In [ ]:
simple_ensemble = pd.read_excel('simple_ensemble_all_mae.xlsx')

In [ ]:
simple_ensemble.head()

In [ ]:
simple_ensemble

# Tạo DF So sánh Simple với Deep learning, plot chart

In [ ]:
# simple_ensemble = df_simple_mix_all_metrix
simple_ensemble = simple_ensemble.rename(columns={"RMSE_Simple_Avg": 'RMSE_1-month',"MAE_Simple_Avg": 'MAE_1-month'})
simple_ensemble.head()
deep_and_simple = pd.concat([     
    result_cnn,
    result_lstm,
    result_tf,
    result_lstm_att,
    simple_ensemble
])
deep_and_simple

In [ ]:
ax = sns.boxplot(y='Alg_name', x='MAE_1-month', data=deep_and_simple)
plt.ylabel('Model')
plt.xlabel('MAE 1-month')
plt.show()

# Rank và đếp số lần hạng 1,2,3 của từng bộ giải thuật

In [ ]:
pivot_deep_and_simple = deep_and_simple.pivot(index='City', columns='Alg_name', values='MAE_1-month').reset_index(drop=True)
ranked_deep_and_simple = pivot_deep_and_simple.rank(axis=1)
ranked_deep_and_simple.mean(axis=0)

In [ ]:
ranked_deep_and_simple.mean(axis=0).to_excel('rank_means_simple_vs_single.xlsx')

In [ ]:
# tp_ranked_deep_and_simple = ranked_deep_and_simple.transpose()
# # tp_ranked_pivot_simple.reset_index("Model Name", inplace = True)
# tp_ranked_deep_and_simple['Mean_Rank']= tp_ranked_deep_and_simple.mean(axis=1)
# rank_sort_simple = tp_ranked_deep_and_simple.sort_values(by = 'Mean_Rank', ascending=True)
# top_rank_simple

In [ ]:
# ranked_deep_and_simple.hist()
ax = ranked_deep_and_simple.plot.hist(bins=10, alpha=0.4)

In [ ]:
sns.histplot(data=ranked_deep_and_simple)

In [ ]:
melted_ranked_deep_and_simple = pd.melt(ranked_deep_and_simple)
melted_ranked_deep_and_simple
ax = sns.boxplot(x='Alg_name', y='value', data=melted_ranked_deep_and_simple)
# plt.setp(ax.get_xticklabels())
plt.xlabel('Model')
plt.ylabel('Rank')

In [ ]:
df_simple = ranked_deep_and_simple['Simple Ensemble'].value_counts()
df_lstm_attention = ranked_deep_and_simple['lstm_attention'].value_counts()
df_lstm = ranked_deep_and_simple['lstm'].value_counts()
df_cnn = ranked_deep_and_simple['cnn'].value_counts()
df_tf = ranked_deep_and_simple['transformers'].value_counts()

arr_rank = []
for i in range(1,6):
    try:
        a = df_simple[i]
    except KeyError:
        a = 0
    
    try:
        b = df_lstm[i]
    except KeyError:
        b = 0
    
    try:
        c = df_lstm_attention[i]
    except KeyError:
        c = 0
    try:
        d = df_cnn[i]
    except KeyError:
        d = 0
    try:
        e = df_tf[i]
    except KeyError:
        e = 0
    arr_rank.append([a,b,c,d,e])
df_rank_table = pd.DataFrame(data=arr_rank, columns=['Simple Ensemble','LSTM','ATT','CNN', 'TF'])
df_rank_table

In [ ]:
df_rank_table.to_excel('count_rank_simple.xlsx')

In [ ]:
pivot_deep_and_simple = deep_and_simple.pivot(index='City', columns='Alg_name', values='RMSE_1-month').reset_index(drop=True)
ranked_deep_and_simple = pivot_deep_and_simple.rank(axis=1)
ranked_deep_and_simple.mean(axis=0)

In [ ]:
melted_ranked_deep_and_simple = pd.melt(ranked_deep_and_simple)
melted_ranked_deep_and_simple
ax = sns.boxplot(x='Alg_name', y='value', data=melted_ranked_deep_and_simple)
# plt.setp(ax.get_xticklabels())
plt.xlabel('Model')
plt.ylabel('Rank')

In [ ]:
df_simple = ranked_deep_and_simple['Simple Ensemble'].value_counts()
df_lstm_attention = ranked_deep_and_simple['lstm_attention'].value_counts()
df_lstm = ranked_deep_and_simple['lstm'].value_counts()
df_cnn = ranked_deep_and_simple['cnn'].value_counts()
df_tf = ranked_deep_and_simple['transformers'].value_counts()

arr_rank = []
for i in range(1,6):
    try:
        a = df_simple[i]
    except KeyError:
        a = 0
    
    try:
        b = df_lstm[i]
    except KeyError:
        b = 0
    
    try:
        c = df_lstm_attention[i]
    except KeyError:
        c = 0
    try:
        d = df_cnn[i]
    except KeyError:
        d = 0
    try:
        e = df_tf[i]
    except KeyError:
        e = 0
    arr_rank.append([a,b,c,d,e])
df_rank_table = pd.DataFrame(data=arr_rank, columns=['Simple Ensemble','LSTM','ATT','CNN', 'TF'])
# df_rank_table.to_excel('count_rank_simple.xlsx')
df_rank_table

In [ ]:
send_to_telegram('ensemble finished')